## Initial exploratory analysis

In [65]:
## importing libraries
import pandas as pd
import numpy as np
import geopy.distance 
from dateutil.relativedelta import relativedelta
from datetime import datetime
import os

## Here we set the directory to look up the folder containing the data
os.chdir("/Users/gabrielmedeiros/Documents/OneDrive/Business analytics/DATA_445_Machine_Learning")


## Reading csv file, index_col = 0 makes the first column of the data to become the index of our pandas data frame
test_data = pd.read_csv('fraudTest.csv', index_col = 0)

train_data = pd.read_csv('fraudTrain.csv', index_col = 0)

n = test_data.shape[0]

/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Head of Train data

In [66]:
print(train_data.shape)
train_data.head()

(1296675, 22)


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


## Head of Test Data

In [ ]:
print(test_data.shape)
test_data.head()

## Feature Engineering

#### Distance

In [ ]:
## Here assign to n the amount of observations we have in the dataset
n = test_data.shape[0]

## Creating empty list to store the reuslts
distance_to_append = []

## Looping through each row an computing the distance between ?????????
for i in range(0,n):
    
    ## Here we gather the lat and long from the ?????
    coords_1 = (test_data['lat'][i], test_data['long'][i])
    
    ## Here we gather the lat and long from the ?????
    coords_2 = (test_data['merch_lat'][i], test_data['merch_long'][i])
    
    ## Here we compute the disance in miles between the locations
    distance_to_append.append(geopy.distance.geodesic(coords_1, coords_2).miles)

## Adding results to our data set
test_data['Distance'] = distance_to_append

In [ ]:
## Visualizing the results
test_data['Distance'].head()

#### Age

In [ ]:
## Creating empty list to store the resutls
ages_to_append = []

## Looping through each observation and computing the age of each individual from its DOB
for i in range(0,n):
    
    ## Here we add the last date of this year
    year_of_2021 = datetime.strptime('2021-12-31', "%Y-%m-%d")
    
    ## Here we call each DOB in the dataset
    dob = datetime.strptime(test_data.dob[i], "%Y-%m-%d")
    
    ## Here we compute the ages
    ages_to_append.append(relativedelta(year_of_2021, dob).years)

## Adding results to our data set
test_data['Age'] = ages_to_append

In [ ]:
## Visualizing the results
test_data['Age'].head()

#### Day of the week

In [ ]:
dates = pd.to_datetime(test_data['trans_date_trans_time'])
test_data['DayOfWeek'] = dates.dt.day_name()

In [ ]:
test_data['DayOfWeek'].head()

#### Month of the year

In [ ]:
months = pd.to_datetime(test_data['trans_date_trans_time'])
test_data['Month'] = dates.dt.month

#### Hour of the day

In [ ]:
hour_of_the_day = []

for i in range(0,n):
    hour_of_the_day.append(datetime.strptime(test_data.trans_date_trans_time[i] ,"%Y-%m-%d %H:%M:%S").hour)
    
test_data['HourOfTheDay'] = hour_of_the_day

In [ ]:
test_data['HourOfTheDay'].head()

In [ ]:
Uses = pd.DataFrame(test_data['cc_num'].value_counts())

Uses['cc_number2'] = Uses.index

Uses = Uses.reset_index(drop = True)

Uses.columns = ['TotalUses','cc_num']

test_data = Uses.merge(test_data, on = 'cc_num', how = 'left')

## Difference in time by minutes for each card in the dataset

In [2]:
new_time = []

for i in range(0,n):
    new_time.append(datetime.strptime(test_data.trans_date_trans_time[i] ,"%Y-%m-%d %H:%M:%S"))
test_data['transformed_time'] = new_time

In [ ]:
test_data['DiffByCardTrans'] = test_data.groupby('cc_num')['transformed_time'].diff().apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')

## Average amount

In [ ]:
amt_per_category = pd.DataFrame(test_data.groupby(['cc_num','category'])['amt'].mean())

amt_per_category['columns'] = amt_per_category.index

amt_per_category = amt_per_category.reset_index().drop(columns = 'columns')

amt_per_category.columns = ['cc_num','category','avg_by_category']

test_data = amt_per_category.merge(test_data, on = 'cc_num', how = 'left')

In [ ]:
avg_amt = pd.DataFrame(test_data.groupby('cc_num')['amt'].mean())

avg_amt['columns'] = avg_amt.index

avg_amt = avg_amt.reset_index().drop(columns = 'columns')

avg_amt.columns = ['cc_num', 'avg_amt']

test_data = avg_amt.merge(test_data, on = 'cc_num', how = 'left')

In [8]:
test_data.groupby('cc_num')['transformed_time'].max()

cc_num
60416207185           2020-12-31 05:16:50
60422928733           2020-12-31 22:18:14
60423098130           2020-12-31 01:48:00
60427851591           2020-12-30 20:45:53
60487002085           2020-12-31 21:41:57
                              ...        
4958589671582726883   2020-12-30 16:02:06
4973530368125489546   2020-12-31 09:05:52
4980323467523543940   2020-12-31 21:34:29
4989847570577635369   2020-12-31 23:31:09
4992346398065154184   2020-12-31 13:24:36
Name: transformed_time, Length: 924, dtype: datetime64[ns]

In [55]:
temp_cc_time_max = pd.DataFrame(test_data.groupby('cc_num')['trans_date_trans_time'].max())
temp_cc_time_min = pd.DataFrame(test_data.groupby('cc_num')['trans_date_trans_time'].min())



temp_cc_time_max['columns'] = temp_cc_time_max.index
temp_cc_time_min['columns'] = temp_cc_time_min.index

temp_cc_time_max = temp_cc_time_max.reset_index().drop(columns = 'columns')
temp_cc_time_min = temp_cc_time_min.reset_index().drop(columns = 'columns')


temp_cc_time_max.columns = ['cc_num','max_date']
temp_cc_time_min.columns = ['cc_num','min_date']


test_data = temp_cc_time_max.merge(test_data, on = 'cc_num', how = 'left')
test_data = temp_cc_time_min.merge(test_data, on = 'cc_num', how = 'left')

In [58]:
min_dates = pd.to_datetime(test_data.min_date) 
max_dates = pd.to_datetime(test_data.max_date)

diff_in_time = max_dates - min_dates
 
temp_data = pd.DataFrame(diff_in_time)
 
test_data['diff_first_last'] = temp_data.apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')

In [64]:
test_data['diff_first_last'].describe()

count    555719.000000
mean     277860.166098
std        4778.937051
min         995.000000
25%      277797.000000
50%      278186.000000
75%      278394.000000
max      278614.000000
Name: diff_first_last, dtype: float64

In [ ]:
## Here we export the file containing all of the new feature to a csv file. 
#test_data.to_csv ("/Users/gabrielmedeiros/Documents/OneDrive/Business analytics/DATA_445_Machine_Learning/FraudTestData.csv", index = None, header=True)